In [ ]:
import numpy as np
import pandas as pd
from support import load_rates
import pylab as pl
from scipy import stats

rates = load_rates()
ftc = rates['ftc'].eval('count/time').rename('ftc')
sr = rates['sr'].eval('count/time').rename('sr')

f, axes = pl.subplots(2, 2, figsize=(10, 10))

In [ ]:
x = ftc.unstack('pupil')
n = len(x)
i = np.arange(n)
np.random.shuffle(i)


In [ ]:
ax = axes[0, 0]
bf = ftc.groupby(['cellid', 'pupil']).apply(lambda x: x.idxmax()[-1])
lb, ub = bf.min(), bf.max()
bf = bf.unstack('pupil')
ax.plot(bf[0], bf[1], 'ko')
ax.plot([lb, ub], [lb, ub], 'r-')
s = stats.wilcoxon(bf[0], bf[1])
t = 'BF (freq with max rate)'
ax.set_title(f'{t}\np = {s.pvalue:.4f}')
ax.set_xlabel('Small pupil')
ax.set_xlabel('Large pupil')

In [ ]:
def center_of_mass(x):
    x = x.rename('ftc').reset_index()
    try:
        return np.average(x['frequency'], weights=x['ftc'])
    except:
        return np.nan
    
ax = axes[0, 1]
bf_mass = ftc.groupby(['cellid', 'pupil']).apply(center_of_mass)
lb, ub = bf_mass.min(), bf_mass.max()
bf_mass = bf_mass.unstack('pupil')
ax.plot(bf_mass[0], bf_mass[1], 'ko')
ax.plot([lb, ub], [lb, ub], 'r-')
s = stats.wilcoxon(bf_mass[0], bf_mass[1])
t = 'BF (center of mass)'
ax.set_title(f'{t}\np = {s.pvalue:.4f}')
ax.set_xlabel('Small pupil')
ax.set_ylabel('Large pupil')

In [ ]:
ax = axes[1, 0]
x = ftc.reset_index().join(sr, on=sr.index.names).set_index(ftc.index.names)
x['gain'] = np.abs(x['ftc']-x['sr'])
gain = x.groupby(['cellid', 'pupil'])['gain'].apply(lambda x: x.max())
lb, ub = gain.min(), gain.max()
gain = gain.unstack('pupil')
ax.plot(gain[0], gain[1], 'ko')
ax.plot([lb, ub], [lb, ub], 'r-')
s = stats.wilcoxon(gain[0], gain[1])
t = 'Gain (max_rate-sr)'
ax.set_title(f'{t}\np = {s.pvalue:.4f}')
ax.set_xlabel('Small pupil')
ax.set_ylabel('Large pupil')

In [ ]:
ax = axes[1, 1]
x = ftc.reset_index().join(sr, on=sr.index.names).set_index(ftc.index.names)
x['abs_delta'] = np.abs(x['ftc'] - x['sr'])
bw = x.groupby(['cellid', 'pupil']).apply(lambda x: np.sum(x['abs_delta'] >= 0.5*x['ftc'].max()))
lb, ub = bw.min(), bw.max()
bw = bw.unstack('pupil')
ax.plot(bw[0], bw[1], 'ko')
ax.plot([lb, ub], [lb, ub], 'r-')
s = stats.wilcoxon(bw[0], bw[1])
t = 'Bandwidth (# of bins with abs(rate-sr) >= 0.5*max_rate)'
ax.set_title(f'{t}\np = {s.pvalue:.4f}')
ax.set_xlabel('Small pupil')
ax.set_ylabel('Large pupil')

In [ ]:
f.tight_layout()
display(f)